# 넘파이를 이용한 고유값->고유벡터 구하기

계산법은 노트

In [ ]:
import numpy as np

a=np.array([[4,1],[2,3]])
#넘파이.선형대수.고유
eigenvalue,eigenvector=np.linalg.eig(a)
print("고유값:",eigenvalue)
print("고유벡터:",eigenvector)

고유값: [5. 2.]
고유벡터: [[ 0.70710678 -0.4472136 ]
 [ 0.70710678  0.89442719]]


# ai에서 얼굴인식을 어떻게? 를 통해 알아보는 고유값 고유벡터

고유값 고유벡터를 구한다=>차원축소가 하고싶어!

PCA 주성분분석을 통해서 얼굴인식

In [ ]:
import numpy as np
import pandas as pd
#재구성 기법중 PCA만
from sklearn.decomposition import PCA # 주 성분 분석 

#3명의 사람이 있다 A1, A2, A3 9x1
A1=np.array([[120,130,125],
            [125,140,135],
            [110,125,130]]).reshape(-1,1) #-1은 나머지 기반으로 알아서 결정하라는 뜻

A2=np.array([[100,110,105],
            [105,120,115],
            [110,115,130]]).reshape(-1,1)

A3=np.array([[140,150,145],
            [135,135,135],
            [125,175,130]]).reshape(-1,1)

#평균 얼굴 계산 9x1
mean_face=(A1+A2+A3)/3

#사람당 특징 추출 9x1
D1=A1-mean_face
D2=A2-mean_face
D3=A3-mean_face

#통합
D=np.hstack([D1,D2,D3])#9x3

#공분산행렬을 구하기 9x9 9개의 픽셀의 모든 다른 각각 픽셀과의 상관관계가 담김
C=np.cov(D)

#고유값, 고유벡터를 구해서 3차원으로 축소
#이를 PCA(주성분분석)이라고 한다.
#사이킷런에 PCA 함수가 구현되어있음

#가장 좋은 3개 찾고  svd_solver라는 방법을 쓰되 다 써서 시드는 42로
pca=PCA(n_components=3,svd_solver='full',random_state=42)
D_T=D.T

pca.fit(D_T)
V_reduced=pca.components_.T # 9x3 PCA로 구한 주성분 3x9를 transpose한다

#새로운 축으로 투영해야함
P_reduced=pca.transform(D_T)
A_new=np.array([[125,155,130],
                [120,115,120],
                [115,150,135]]).reshape(-1,1)
D_new=(A_new-mean_face).reshape(-1) #자동으로 1차원배열로

P_new=pca.transform([D_new])[0] #새로운 얼굴특징 D_new를 PCA주성분 축에 투영하자

#pca.transform입력이 2차원 배열이 디폴트임.. 첫 번째 배열 샘플만 가지고 오기 위해서 [0]
# 유클리드 거리 놈 nom = 벡터 길이 구하는 함수
distance=np.linalg.norm(P_reduced-P_new,axis=1) #axis=1은 각 행마다 거리 계산
idx=np.argmin(distance)

li=['A1','A2','A3']
print("신규 사람은",li[idx],"와 유사한 사람")



[[-5.62306418e+00 -1.72092686e+01 -7.67587330e-16]
 [-4.75414469e+01  1.00459913e+01 -6.56565028e-16]
 [ 5.31645111e+01  7.16327736e+00  4.53657997e-16]]
신규 사람은 A3 와 유사한 사람
